In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7744,2021-02-26T17:00:00,ITA,19,Sicilia,38.115697,13.362357,776,132,908,25689,...,NaN,8.0,NaN,NaN,151587.0,0.0,1731142.0,660971.0,ITG,ITG1
7745,2021-02-26T17:00:00,ITA,9,Toscana,43.769231,11.255889,844,163,1007,15876,...,NaN,14.0,NaN,NaN,153418.0,623.0,2434806.0,250976.0,ITI,ITI1
7746,2021-02-26T17:00:00,ITA,10,Umbria,43.106758,12.388247,441,78,519,7809,...,"Si fa presente che 16 dei ricoveri NON UTI, no...",7.0,NaN,NaN,44241.0,0.0,688407.0,114011.0,ITI,ITI2
7747,2021-02-26T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,8,1,9,132,...,NaN,0.0,NaN,NaN,8011.0,0.0,73849.0,3155.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7744,2021-02-26T17:00:00,19,Sicilia,776,132,908,25689,26597,-429,578,...,151587,2392113,1120327.0,8.0,151587.0,0.0,1731142.0,660971.0,ITG,ITG1
7745,2021-02-26T17:00:00,9,Toscana,844,163,1007,15876,16883,753,1254,...,154041,2685782,1420746.0,14.0,153418.0,623.0,2434806.0,250976.0,ITI,ITI1
7746,2021-02-26T17:00:00,10,Umbria,441,78,519,7809,8328,6,331,...,44241,802418,312553.0,7.0,44241.0,0.0,688407.0,114011.0,ITI,ITI2
7747,2021-02-26T17:00:00,2,Valle d'Aosta,8,1,9,132,141,-1,13,...,8011,77004,43899.0,0.0,8011.0,0.0,73849.0,3155.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-02-26', '2021-02-25')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-02-26T17:00:00,1,Piemonte,1938,163,2101,13022,15123,800,1526,...,245044,2537151,1242421.0,9.0,239982.0,5062.0,2060229.0,476922.0,ITC,ITC1
1,2021-02-26T17:00:00,2,Valle d'Aosta,8,1,9,132,141,-1,13,...,8011,77004,43899.0,0.0,8011.0,0.0,73849.0,3155.0,ITC,ITC2
2,2021-02-26T17:00:00,3,Lombardia,4034,416,4450,56817,61267,3439,4557,...,595361,6566513,3092577.0,38.0,587855.0,7506.0,6185743.0,380770.0,ITC,ITC4
3,2021-02-26T17:00:00,5,Veneto,784,106,890,22549,23439,490,1174,...,331451,4970754,1392786.0,8.0,327571.0,3880.0,4022126.0,948628.0,ITH,ITH3
4,2021-02-26T17:00:00,6,Friuli Venezia Giulia,348,56,404,9471,9875,235,468,...,76003,1337295,476513.0,6.0,71008.0,4995.0,1243380.0,93915.0,ITH,ITH4
5,2021-02-26T17:00:00,7,Liguria,491,52,543,4640,5183,108,351,...,77379,1021703,435483.0,3.0,77379.0,0.0,919627.0,102076.0,ITC,ITC3
6,2021-02-26T17:00:00,8,Emilia-Romagna,2092,211,2303,36919,39222,1529,2575,...,255479,3794731,1488902.0,27.0,255361.0,118.0,3376900.0,417831.0,ITH,ITH5
7,2021-02-26T17:00:00,9,Toscana,844,163,1007,15876,16883,753,1254,...,154041,2685782,1420746.0,14.0,153418.0,623.0,2434806.0,250976.0,ITI,ITI1
8,2021-02-26T17:00:00,10,Umbria,441,78,519,7809,8328,6,331,...,44241,802418,312553.0,7.0,44241.0,0.0,688407.0,114011.0,ITI,ITI2
9,2021-02-26T17:00:00,11,Marche,561,81,642,9102,9744,134,686,...,66471,821182,498803.0,8.0,66471.0,0.0,759901.0,61281.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-02-25T17:00:00,1,Piemonte,1909,162,2071,12252,14323,706,1454,...,243518,2511427,1233224.0,16.0,238693.0,4825.0,2051468.0,459959.0,ITC,ITC1
1,2021-02-25T17:00:00,2,Valle d'Aosta,10,1,11,131,142,-4,2,...,7998,76738,43783.0,0.0,7998.0,0.0,73623.0,3115.0,ITC,ITC2
2,2021-02-25T17:00:00,3,Lombardia,4024,407,4431,53397,57828,2603,4243,...,590804,6519788,3077516.0,30.0,583731.0,7073.0,6151289.0,368499.0,ITC,ITC4
3,2021-02-25T17:00:00,5,Veneto,793,106,899,22050,22949,617,1304,...,330277,4932271,1389455.0,15.0,326690.0,3587.0,4008425.0,923846.0,ITH,ITH3
4,2021-02-25T17:00:00,6,Friuli Venezia Giulia,335,56,391,9249,9640,323,683,...,75535,1328704,473707.0,4.0,70714.0,4821.0,1237507.0,91197.0,ITH,ITH4
5,2021-02-25T17:00:00,7,Liguria,493,52,545,4530,5075,168,452,...,77028,1015161,432933.0,2.0,77028.0,0.0,915980.0,99181.0,ITC,ITC3
6,2021-02-25T17:00:00,8,Emilia-Romagna,2058,203,2261,35432,37693,922,2090,...,252915,3754583,1481936.0,21.0,252796.0,119.0,3355208.0,399375.0,ITH,ITH5
7,2021-02-25T17:00:00,9,Toscana,853,159,1012,15118,16130,725,1374,...,152787,2662754,1408946.0,12.0,152199.0,588.0,2420676.0,242078.0,ITI,ITI1
8,2021-02-25T17:00:00,10,Umbria,448,77,525,7797,8322,-95,267,...,43910,796251,311511.0,5.0,43910.0,0.0,684692.0,111559.0,ITI,ITI2
9,2021-02-25T17:00:00,11,Marche,559,84,643,8967,9610,247,732,...,65785,815619,495328.0,4.0,65785.0,0.0,755495.0,60124.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-02-26T17:00:00,1,Piemonte,1938,163,2101,13022,15123,800,1526,...,2537151,1242421.0,9.0,239982.0,5062.0,2060229.0,476922.0,ITC,ITC1,4.341375e+06
1,2021-02-26T17:00:00,2,Valle d'Aosta,8,1,9,132,141,-1,13,...,77004,43899.0,0.0,8011.0,0.0,73849.0,3155.0,ITC,ITC2,1.255010e+05
2,2021-02-26T17:00:00,3,Lombardia,4034,416,4450,56817,61267,3439,4557,...,6566513,3092577.0,38.0,587855.0,7506.0,6185743.0,380770.0,ITC,ITC4,1.010397e+07
3,2021-02-26T17:00:00,5,Veneto,784,106,890,22549,23439,490,1174,...,4970754,1392786.0,8.0,327571.0,3880.0,4022126.0,948628.0,ITH,ITH3,4.907704e+06
4,2021-02-26T17:00:00,6,Friuli Venezia Giulia,348,56,404,9471,9875,235,468,...,1337295,476513.0,6.0,71008.0,4995.0,1243380.0,93915.0,ITH,ITH4,1.211357e+06
5,2021-02-26T17:00:00,7,Liguria,491,52,543,4640,5183,108,351,...,1021703,435483.0,3.0,77379.0,0.0,919627.0,102076.0,ITC,ITC3,1.543127e+06
6,2021-02-26T17:00:00,8,Emilia-Romagna,2092,211,2303,36919,39222,1529,2575,...,3794731,1488902.0,27.0,255361.0,118.0,3376900.0,417831.0,ITH,ITH5,4.467118e+06
7,2021-02-26T17:00:00,9,Toscana,844,163,1007,15876,16883,753,1254,...,2685782,1420746.0,14.0,153418.0,623.0,2434806.0,250976.0,ITI,ITI1,3.722729e+06
8,2021-02-26T17:00:00,10,Umbria,441,78,519,7809,8328,6,331,...,802418,312553.0,7.0,44241.0,0.0,688407.0,114011.0,ITI,ITI2,8.802850e+05
9,2021-02-26T17:00:00,11,Marche,561,81,642,9102,9744,134,686,...,821182,498803.0,8.0,66471.0,0.0,759901.0,61281.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-02-26T17:00:00,1,Piemonte,1938,163,2101,13022,15123,800,1526,...,476922.0,ITC,ITC1,4.341375e+06,Piemonte,1938,5824,163,628,99
1,2021-02-26T17:00:00,2,Valle d'Aosta,8,1,9,132,141,-1,13,...,3155.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,8,227,1,20,12
2,2021-02-26T17:00:00,3,Lombardia,4034,416,4450,56817,61267,3439,4557,...,380770.0,ITC,ITC4,1.010397e+07,Lombardia,4034,10435,416,1416,114
3,2021-02-26T17:00:00,5,Veneto,784,106,890,22549,23439,490,1174,...,948628.0,ITH,ITH3,4.907704e+06,Veneto,784,6000,106,1000,0
4,2021-02-26T17:00:00,6,Friuli Venezia Giulia,348,56,404,9471,9875,235,468,...,93915.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,348,1277,56,175,0
5,2021-02-26T17:00:00,7,Liguria,491,52,543,4640,5183,108,351,...,102076.0,ITC,ITC3,1.543127e+06,Liguria,491,1795,52,216,18
6,2021-02-26T17:00:00,8,Emilia-Romagna,2092,211,2303,36919,39222,1529,2575,...,417831.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,2092,6126,211,757,0
7,2021-02-26T17:00:00,9,Toscana,844,163,1007,15876,16883,753,1254,...,250976.0,ITI,ITI1,3.722729e+06,Toscana,844,5033,163,576,24
8,2021-02-26T17:00:00,10,Umbria,441,78,519,7809,8328,6,331,...,114011.0,ITI,ITI2,8.802850e+05,Umbria,441,851,78,141,2
9,2021-02-26T17:00:00,11,Marche,561,81,642,9102,9744,134,686,...,61281.0,ITI,ITI3,1.518400e+06,Marche,561,1176,81,233,40


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-02-26T17:00:00,3,Lombardia,4034,416,4450,56817,61267,3439,4557,...,29.38,3420,314,46725,15061.0,47,1071,9.75,0.04510,5.89
1,2021-02-26T17:00:00,8,Emilia-Romagna,2092,211,2303,36919,39222,1529,2575,...,27.87,1487,485,40148,6966.0,30,1005,6.41,0.05764,5.72
2,2021-02-26T17:00:00,15,Campania,1294,144,1438,72263,73701,729,2519,...,23.23,727,134,22416,13526.0,11,1779,11.24,0.04354,4.54
3,2021-02-26T17:00:00,12,Lazio,1830,217,2047,33386,35433,438,1539,...,23.01,425,283,34715,18186.0,19,1082,4.43,0.02624,3.95
4,2021-02-26T17:00:00,1,Piemonte,1938,163,2101,13022,15123,800,1526,...,25.96,770,72,25724,9197.0,5,721,5.93,0.03515,5.64
5,2021-02-26T17:00:00,9,Toscana,844,163,1007,15876,16883,753,1254,...,28.30,758,-120,23028,11800.0,22,479,5.45,0.03368,4.14
6,2021-02-26T17:00:00,5,Veneto,784,106,890,22549,23439,490,1174,...,10.60,499,-130,38483,3331.0,23,661,3.05,0.02392,6.75
7,2021-02-26T17:00:00,16,Puglia,1246,164,1410,30629,32039,67,1104,...,28.82,75,-50,8179,4779.0,22,1015,13.50,0.02754,3.61
8,2021-02-26T17:00:00,11,Marche,561,81,642,9102,9744,134,686,...,34.76,135,-46,5563,3475.0,5,547,12.33,0.04518,4.38
9,2021-02-26T17:00:00,13,Abruzzo,614,74,688,12043,12731,96,594,...,36.27,95,-59,8929,3087.0,11,487,6.65,0.04549,4.10


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,78,63,141,55.32
1,P.A. Trento,38,52,90,42.22
2,Molise,16,23,39,41.03
3,Abruzzo,74,130,204,36.27
4,Marche,81,152,233,34.76
5,P.A. Bolzano,34,66,100,34.00
6,Friuli Venezia Giulia,56,119,175,32.00
7,Lombardia,416,1000,1416,29.38
8,Puglia,164,405,569,28.82
9,Toscana,163,413,576,28.30


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))